## Berechnung eines Steinschlags an der Kantonsstrasse in Schiers (Kt. GR) 
   In der folgenden Berechnung wird anhand der Daten der letzten drei Monate die Wahrscheinlichkeit 
   berechnet für einen tödlichen Steinschlag auf der Strasse. Anhand dessen wird ausgesagt 
   ob die Strasse gesperrt werden muss oder nicht.
   Wird die Strasse gesperrt wird untersucht ob diese den ganzen Tag gesperrt bleibt 
   oder nur für eine gewisse Zeit am Tag

### Annahmen:
1. Die Ablösungszonen 1 und 2 befinden sich an zwei verschiedenen Stellen
2. Die Steine aus den beiden Ablösungszonen werden mit dem gleichen Netz aufgefangen
3. Die Wahrscheinlichkeiten werden pro Stundenfenster berechnet und nicht pro Minute
4. Die Verteilung des Verkehrs basiert auf den Erkentnissen des Bundesamtes für Statistik
5. Steinmassen die nicht möglich sind, werden auf die nächsthöhere Skalierung gerundet (Schätzfehler)
6. Die Distanz der Gefahrenzone beträgt 100 m

### Berechnungsmodel:
Um eine Simulations des Models zu bewerkstelligen müssen die einzelnen Variabeln und deren logische Abhängigkeiten 
in eine Reihenfolge gebracht werden. Aus dieser Betrachtung sollte sich dann ein Model ergeben welches durch eine 
Monte Carlo Simulation berechnet werden kann:

1. P (kinetische Energie) = Wahrscheinlichkeit Geschwindigkeit des Steins  * Wahrscheinlich Masse des Steins
2. P (T bis zum nächsten Abschlag) = Wahrscheinlichkeit des Zeitdeltas
3. P (Netz bricht beim ersten Aufprall) = P (kinetische Energie > 1000 kJ)
4. P1 (Netz bricht über Zeit) = Wenn Summe Masse der Steine > 2000 kg, dann P (kinetische Energie >= 500 kJ )
4. P2 (Netz bricht über Zeit) = Wenn Summe Masse der Steine < 2000 kg, dann P (kinetische Energie >= 1000 kJ )
6. Summe (Auto im Zeitfenster) = Summe der Menge der Autos in den Zeitfenstern (Leerung + P (Zeitdelta)  
7. Aufenthaltsdauer 1 Autos in Gefahrenzone = Distanz / Geschwindigkeit
8. P (Auto wird getroffen) = Verteilung (Minute des Durchbruchs) * Verteilung (Aufenthaltsdauer Fahrzeug) 

Das Zusammensetzen der einzelnen Wahrscheinlichkeiten wird in einer Monte Carlo Simulation durch Zufallsvariablen 
simuliert

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math

## 1. Daten einlesen, säubern und in eine einheitliche Form bringen.

In [2]:
# Daten laden
raw_data_1 = pd.read_csv("out_1.csv")
raw_data_2 = pd.read_csv("out_2.csv")

In [3]:
# Überprüfen wie viele "NA" Werte noch vorhanden sind
raw_data_1.isna().sum()

Datum                    11
Uhrzeit                  11
Masse [kg]               11
Geschwindigkeit [m/s]    11
Unnamed: 4               79
                         ..
Unnamed: 72              79
Unnamed: 73              79
Unnamed: 74              79
Unnamed: 75              79
Unnamed: 76              79
Length: 77, dtype: int64

In [4]:
# Alle Zeilen löschen die in ALLEN Zellen keine Werte haben
data_1_no_rows = raw_data_1.dropna(how='all')
data_2_no_rows = raw_data_2.dropna(how='all')

# Alle Kolonen löschen die NA Werte enthalten
data_1_no_na = data_1_no_rows.dropna(axis= 'columns')
data_2_no_na = data_2_no_rows.dropna(axis= 'columns')

In [5]:
#Um eine Vergleichbarkeit von Daten herstellen zu können sollte die Benennung der Zellen die selbe sein
data_stelle_1= data_1_no_na.rename(columns= {'Datum':'Date',
                                             'Uhrzeit': 'Time',
                                             'Masse [kg]' : 'Masse in kg' ,
                                             'Geschwindigkeit [m/s]' : 'Speed in m/s' 
                                            })

data_stelle_2= data_2_no_na.rename(columns= {'Uhrzeit': 'Time',
                                             'm [kg]' : 'Masse in kg' ,
                                             'v [m/s]' : 'Speed in m/s' 
                                            })

In [6]:
# Überprüfen wie viele "NA" Werte noch vorhanden sind nachdem auch die Anpassung der Benennungen durchgeführt worden ist
data_stelle_1.isna().sum()

Date            0
Time            0
Masse in kg     0
Speed in m/s    0
dtype: int64

In [7]:
data_stelle_2.isna().sum()

Date            0
Time            0
Masse in kg     0
Speed in m/s    0
dtype: int64

In [8]:
# Zeit und Datum in eine Spalte bringen und in ein Datetime Object transformieren
data_stelle_1 ['Datetime'] = pd.to_datetime(data_stelle_1 ['Date'] + " " + data_stelle_1 ['Time'])
data_stelle_2 ['Datetime'] = pd.to_datetime(data_stelle_2 ['Date'] + " " + data_stelle_2 ['Time'])

In [9]:
# Die beiden Spalten 'Date' und 'Time' werden nicht mehr benötigt und werden gelöscht
data_stelle_1 = data_stelle_1.drop('Time', axis=1)
data_stelle_1 = data_stelle_1.drop('Date', axis=1)

data_stelle_2 = data_stelle_2.drop('Time', axis=1)
data_stelle_2 = data_stelle_2.drop('Date', axis=1)

In [10]:
# Datetime nach Zeit sortieren
data_stelle_2.sort_values(by= ['Datetime'])
data_stelle_1.sort_values(by= ['Datetime'])
   

,Masse in kg,Speed in m/s,Datetime
0,194.0,8.4,2019-01-01 09:00:00
1,224.0,8.8,2019-01-01 21:00:00
2,3104.0,9.2,2019-01-02 14:00:00
3,228.0,8.0,2019-01-04 15:00:00
4,755.0,7.0,2019-01-05 23:00:00
...,...,...,...
63,167.0,8.9,2019-03-18 16:00:00
64,2847.0,7.0,2019-03-22 18:00:00
65,44.0,8.9,2019-03-26 00:00:00
66,45.0,8.4,2019-03-26 06:00:00


## 2. Hinzufügen und Berechnung von fehlenden Daten

In [11]:
# Berechnung der Zeit zwischen den einzelnen Steinschlägen. 
# Die Anzahl Differenzen beläuft sich auf Anzahl Beobachtungen - 1, da der Startpunkt nicht berücksichtig werden kann

# Zeitdifferenzen Abschlagstelle 1
t_delta_stelle_1 = []
for x in range (len(data_stelle_1['Datetime'])):
    try:
        delta = int(abs(data_stelle_1 ['Datetime'] [x] - data_stelle_1 ['Datetime'] [x+1]).total_seconds() / 3600)
        t_delta_stelle_1.append(delta)   
    except:
        pass   

# Zeitdifferenzen Abschlagstelle 2
t_delta_stelle_2 = []
for x in range (len(data_stelle_2['Datetime'])):
    try:
        delta = int(abs(data_stelle_2 ['Datetime'] [x] - data_stelle_2 ['Datetime'] [x+1]).total_seconds() / 3600)
        t_delta_stelle_2.append(delta)   
    except:
        pass   
    
# Liste in Pandas DataFrame Objekt transformieren
t_delta_stelle_1 = pd.DataFrame(t_delta_stelle_1, columns = ['Delta'])
t_delta_stelle_2 = pd.DataFrame(t_delta_stelle_2, columns = ['Delta'])

In [12]:
# Die Masse der Steine muss immer > 0 sein, da die Masse im vornherein 
# geschätzt wurde können 0 KG Steine aufgerundet werden 
data_stelle_2 ['Datetime'] .where(data_stelle_2 ['Masse in kg'] == 0)

# Wie zu sehen ist am 2019-03-10 16:00:00 (Zeile 23) ein Stein mit der Masse 0 Kg ins Netz gefallen. Diese Masse wird auf 1 kg angepasst
data_stelle_2.at [23, 'Masse in kg'] = 1.0

In [48]:
#Verteilung des Verkehrs über die Zeit. Dafür werden die Daten des Bundesamts für Statstik herangezogen.
#Credit geht hier an Roman Studer, der die Daten vom Bundesamt für Statistik besorgt hat

traffic = pd.read_excel("traffic.xlsx")

# Normierungfaktors um die percentile auf 1 zu bringen
traffic_factor = 1 / (traffic['percentile'].sum() / 100) 

# Multiplikation der Kolone um die Normierung zu erhalten
traffic['percentile'] *= traffic_factor


,hour,percentile
0,0,0.694661
1,1,0.328086
2,2,0.210446
3,3,0.194323
4,4,0.451570
5,5,1.564517
6,6,4.248568
7,7,5.640588
8,8,5.126819
9,9,5.531887


## 3. Datenanalyse - Welche Verteilung liegt in den einzelnen Datensätzen vor?
Vorgehen: Zuerst werden Histogramme der einzelnen Variabeln erstellet und im zweiten Schritt ein QQ Plot.   
Um zu überprüfen welche Verteilung den vorhergegangenen Histogrammen zugrunde liegt, 
werden im nächsten Schritt QQ-Plots erstellt um eine visuelle Überprüfung einer Normalverteilung vorzunehmen.

#### Masse Ablösestelle 1

In [ ]:
# Analyse 
data_stelle_1['Masse in kg'].describe()

In [ ]:
# Histogram 
plt.figure(figsize=(8, 8), dpi=80)
plt.hist(data_stelle_1['Masse in kg'], bins = 20)

plt.title("Histogramm der Masse - Stelle 1")
plt.ylabel("Anzahl Ablösungen")
plt.xlabel("Masse in kg")   
plt.grid(True)
plt.show()

In [ ]:
# QQ - Plot
plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(data_stelle_1['Masse in kg'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Masse Stelle 1")
plt.grid(True)
plt.show()

#### Masse Ablösestelle 2

In [ ]:
# Analyse
data_stelle_2['Masse in kg'].describe()

In [ ]:
# Histogram 
plt.figure(figsize=(8, 8), dpi=80)
plt.hist(data_stelle_2['Masse in kg'], bins = 15)

plt.title("Histogramm der Masse - Stelle 2")
plt.ylabel("Anzahl Ablösungen")
plt.xlabel("Masse in kg") 
plt.grid(True)
plt.show()

In [ ]:
# QQ - Plot
plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(data_stelle_2['Masse in kg'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Masse Stelle 2")
plt.grid(True)
plt.show()

#### Geschwindigkeit Ablösestelle 1

In [ ]:
# Analyse
data_stelle_1['Speed in m/s'].describe()

In [ ]:
# Histogram 
plt.figure(figsize=(8, 8), dpi=80)
plt.hist(data_stelle_1['Speed in m/s'], bins = 22)

plt.title("Histogramm der Geschwindigkeit - Stelle 1")
plt.ylabel("Anzahl Ablösungen")
plt.xlabel("Speed in m/s")   
plt.grid(True)
plt.show()

In [ ]:
# QQ - Plot
plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(data_stelle_1['Speed in m/s'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Speed in m/s Stelle 1")
plt.grid(True)
plt.show()

#### Geschwindigkeit Ablösestelle 2

In [ ]:
# Analyse 
data_stelle_2['Speed in m/s'].describe()

In [ ]:
# Histogram 
plt.figure(figsize=(8, 8), dpi=80)
plt.hist(data_stelle_2['Speed in m/s'], bins = 20)

plt.title("Histogramm der Geschwindigkeit - Stelle 2")
plt.ylabel("Anzahl Ablösungen")
plt.xlabel("Speed in m/s")
plt.grid(True)
plt.show()

In [ ]:
# QQ - Plot
plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(data_stelle_1['Speed in m/s'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Speed in m/s Stelle 2")
plt.grid(True)
plt.show()

#### Zeitdifferenzen Ablösestelle 1

In [ ]:
# Analyse 
t_delta_stelle_1.describe()

In [ ]:
# Histogram 
plt.figure(figsize=(8, 8), dpi=80)
plt.hist(t_delta_stelle_1 ['Delta'], bins = 50)


plt.title("Histogramm der Zeitdeltas - Stelle 1")
plt.ylabel("Anzahl Ablösungen")
plt.xlabel("Zeitdifferenzen")
plt.grid(True)
plt.show()

In [ ]:
# QQ - Plot
plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(t_delta_stelle_1['Delta'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Zeitdelta Stelle 1")
plt.grid(True)
plt.show()

#### Zeitdifferenzen Ablösestelle 2

In [ ]:
# Analyse 
t_delta_stelle_2.describe()

In [ ]:
# Histogram
plt.figure(figsize=(8, 8), dpi=80)
plt.hist(t_delta_stelle_2 ['Delta'], bins = 15)

plt.title("Histogramm der Zeitdeltas - Stelle 2")
plt.ylabel("Anzahl Ablösungen")
plt.xlabel("Zeitdifferenzen")
plt.grid(True)
plt.show()

In [ ]:
# QQ - Plot
plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(t_delta_stelle_2['Delta'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Zeitdelta Stelle 2")
plt.grid(True)
plt.show()

## 5. Distributionen der Variabeln Geschwindigkeit, Masse und Zeitdelta fitten 

In [ ]:
# Geschwindigkeit Stelle 1
data_speed_1 = data_stelle_1['Speed in m/s']


plt.figure(figsize=(8, 8), dpi=80)
plt.grid(True)
plt.hist(data_speed_1, bins = 35, density = True)

lnspc = np.linspace(data_speed_1.min(), data_speed_1.max(), len(data_speed_1))


mu_speed_1, s_speed_1 = stats.norm.fit(data_speed_1) # get mean and standard deviation  
pdf_speed_1 = stats.norm.pdf(lnspc, mu_speed_1, s_speed_1) # now get theoretical values in our interval  
plt.plot(lnspc, pdf_speed_1, label='Normal Dist') # plot it

title = "Fit results: Distribution = Norm, mu = %.2f,  std = %.2f" % (mu_speed_1, s_speed_1)
plt.title(title)
plt.show()

In [ ]:
# Geschwindigkeit Stelle 2
data_speed_2 = data_stelle_2['Speed in m/s']

plt.figure(figsize=(8, 8), dpi=80)
plt.grid(True)
plt.hist(data_speed_2, bins = 25, density = True)

lnspc = np.linspace(data_speed_2.min(), (data_speed_2.max()+5), len(data_speed_2))


mu_speed_2, s_speed_2 = stats.norm.fit(data_speed_2) # get mean and standard deviation  
pdf_speed_2 = stats.norm.pdf(lnspc, mu_speed_2, s_speed_2) # now get theoretical values in our interval  
plt.plot(lnspc, pdf_speed_2, label='Normal Dist') # plot it

title = "Fit results: Distribution = Norm, mu = %.2f,  std = %.2f" % (mu_speed_2, s_speed_2)
plt.title(title)
plt.show()

In [ ]:
# Masse Stelle 1
data_masse_1 = data_stelle_1['Masse in kg']

# Histogram 
plt.figure(figsize=(8, 8), dpi=80)
plt.grid(True)
plt.hist(data_masse_1, bins = 30, density = True)

xt = plt.xticks()[0]
xmin, xmax = min(xt), max(xt)  
lnspc = np.linspace(xmin, xmax, len(data_masse_1))

shape_masse_1, loc_masse_1, scale_masse_1 = stats.lognorm.fit(data_masse_1, floc = 0)  # 
pdf_masse_1 = stats.lognorm.pdf(lnspc, shape_masse_1, loc_masse_1, scale_masse_1)  
plt.plot(lnspc, pdf_masse_1)

mu_masse_1 = np.log(scale_masse_1)
s_masse_1 = shape_masse_1

title = "Fit results: Distribution = LogNorm, shape = %.2f, scale = %.2f, location = %.2f " % (shape_masse_1, scale_masse_1, loc_masse_1)
plt.title(title)
plt.show()

In [ ]:
# Masse Stelle 2
data_masse_2 = data_stelle_2['Masse in kg']

plt.figure(figsize=(8, 8), dpi=80)
plt.grid(True)
plt.hist(data_masse_2, bins = 30, density = True)

xt = plt.xticks()[0]
xmin, xmax = min(xt), max(xt)  
lnspc = np.linspace(xmin, xmax, len(data_masse_2))

shape_masse_2, loc_masse_2, scale_masse_2 = stats.lognorm.fit(data_masse_2, floc = 0)  # 
pdf_masse_2 = stats.lognorm.pdf(lnspc, shape_masse_2, loc_masse_2, scale_masse_2)  
plt.plot(lnspc, pdf_masse_2)

mu_masse_2 = np.log(scale_masse_2)
s_masse_2 = shape_masse_2

title = "Fit results: Distribution = LogNorm, shape = %.2f, scale = %.2f, location = %.2f " % (shape_masse_2, scale_masse_2, loc_masse_2)
plt.title(title)
plt.show()

In [ ]:
# Zeitdelta Stelle 1
data_time_1 = t_delta_stelle_1 ['Delta']

plt.figure(figsize=(8, 8), dpi=80)
plt.grid(True)
plt.hist(data_time_1, bins = 30, density = True)

xt = plt.xticks()[0]
xmin, xmax = min(xt), max(xt)  
lnspc = np.linspace(xmin, xmax, len(data_time_1))

alpha_t_delta_1, loc_t_delta_1, beta_t_delta_1 = stats.gamma.fit(data_time_1)  
pdf_t_delta_1 = stats.gamma.pdf(lnspc, alpha_t_delta_1, loc_t_delta_1, beta_t_delta_1)  
plt.plot(lnspc, pdf_t_delta_1)

mu_t_delta_1 = alpha_t_delta_1 * beta_t_delta_1
s_t_delta_1 = math.sqrt(alpha_t_delta_1 * beta_t_delta_1 * beta_t_delta_1)

title = "Fit results:Distribution = Gamma, mu = %.2f,  std = %.2f, " % (mu_t_delta_1, s_t_delta_1)
plt.title(title)
plt.show()

In [ ]:
# Zeitdelta Stelle 2
data_time_2 = t_delta_stelle_2 ['Delta']

plt.figure(figsize=(8, 8), dpi=80)
plt.grid(True)
plt.hist(data_time_2, bins = 20, density = True)

xt = plt.xticks()[0]
xmin, xmax = min(xt), max(xt)  
lnspc = np.linspace(xmin, xmax, len(data_time_2))

alpha_t_delta_2, loc_t_delta_2, beta_t_delta_2 = stats.gamma.fit(data_time_2)  
pdf_t_delta_2 = stats.gamma.pdf(lnspc, alpha_t_delta_2, loc_t_delta_2, beta_t_delta_2)  
plt.plot(lnspc, pdf_t_delta_2)

mu_t_delta_2 = alpha_t_delta_2 * beta_t_delta_2
s_t_delta_2 = math.sqrt(alpha_t_delta_2 * beta_t_delta_2 * beta_t_delta_2)

title = "Fit results: Distribution = Gamma, mu = %.2f,  std = %.2f, " % (mu_t_delta_1, s_t_delta_1)
plt.title(title)
plt.show()

## Erkentnisse der Verteilungen:

1. Geschwindigkeiten: Anhand des QQ Plots ist erkennbar dass sich die 
Geschwindigkeiten auf der 45 Grad Linie sauber verteilen. Bis auf ein paar Unebenheiten kann dieser Schluss zugelassen.
Eine genauere Aussage ist nicht möglich, da der Datensatz zu klein ist.

2. Masse: Dadurch, dass eine negative Beobachtung der Masse nicht vorkommen kann, 
kann im ersten Schritt auf eine lognormierte Verteilung geschlossen werden. 
Dieser Entschluss kann durch den QQ Plot gestützt werden. 
Ein gewölbter QQ Plot in einer solchen Form schliesst auf exponentielle Verteilungen zurück.
Im anschliessenden Schritt wird eine gefittete LogNorm Verteilung über das Histogram gelegt
um eine visuelle Überprüfung durchzuführen. Auch hier ist zu erkennen, 
dass Histogram und Distribution gut zueinander passen. 

3. Zeitdelta: Auf den ersten Blick unterscheiden sich die Histogramme und QQ-Plot des Zeitdeltas 
nicht massiv von denen der Masse. Jedoch ist beim Zeitdelta die Möglichkeit vorhanden, dass dieses 0 beträgt. 
Da eine lognormal Verteilung 0 Werte nicht verabreiten kann, wird auf eine Gamma Verteilung zurückgegriffen.
Bei der visuellen Überprüfung ist zu sehen, dass die Verteilung und das Histogram ziemlich gut zueinander passen.

4. Verkehrsaufkommen: In der Verteilung 

## 6. Zufalsvariabeln definieren + Monte Carlo berechnen

#### Die Motnte Carlo Simulation inkls. Definitionen der Vertielungen sind zu finden in: 

#### "Monte_Carlo_Sim.ipynb"
  

## 7. Berechnung ob ein Fahrzeug getroffen wird wenn das Netz bricht

In [50]:
breakthrough = pd.read_excel('breakthrough.xlsx')

# Aus "breakthrough" brauchen wir nur die Stunde
verkehr

,Unnamed: 0,Unnamed: 0.1,Hours,Day,Hour of Day,M 1,Speed 1,M 2,Speed 2,E in kJ 1,E in kJ 2,Sum M,Breakthrough
0,3479,3479,3474,144,18,0.0,0.00,1693.9,42.21,0.000000,1508.997348,1693.9,1
1,3939,3939,3932,163,20,0.0,0.00,4740.5,34.96,0.000000,2896.923342,6419.7,1
2,19162,19162,19125,796,21,0.0,0.00,2415.6,30.00,0.000000,1087.020000,3418.2,1
3,33692,33692,33621,1400,21,0.0,0.00,3872.3,31.49,0.000000,1919.925257,3872.3,1
4,35864,35864,35786,1491,2,21662.6,11.28,0.0,0.00,1378.157282,0.000000,21662.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,859347,859347,857435,35726,11,0.0,0.00,1575.1,46.83,0.000000,1727.135661,1575.1,1
111,862031,862031,860115,35838,3,0.0,0.00,941.1,48.93,0.000000,1126.564883,941.1,1
112,863499,863499,861576,35899,0,0.0,0.00,1846.5,37.93,0.000000,1328.265834,1846.5,1
113,871887,871887,869945,36247,17,0.0,0.00,604.9,47.41,0.000000,679.819315,8044.9,1
